In [75]:
import findspark

In [76]:

findspark.init("/opt/manual/spark")

In [77]:

from pyspark.sql import SparkSession, functions as F
import pandas as pd

In [78]:
spark = SparkSession.builder \
.appName("arabam_com") \
.master("yarn") \
.getOrCreate()

In [79]:
#! hdfs dfs -rm /user/train/datasets/arabam.csv

In [80]:
#! hdfs dfs -put ~/Desktop/arabam/arabam.csv /user/train/datasets/

In [81]:
df1 = spark.read.format("csv") \
.option("header",True) \
.option("inferShema",True) \
.option("sep",",")  \
.load("hdfs://localhost:9000/user/train/datasets/arabam.csv")

In [82]:
df1.limit(10).toPandas()

,_c0,İlan No:,İlan Tarihi:,Marka:,Seri:,Model:,Yıl:,Yakıt Tipi:,Vites Tipi:,Motor Hacmi:,Motor Gücü:,Kilometre:,car_link:,price
0,0,16251517,23 Aralık 2020,Volkswagen,Passat Variant,1.6 TDi BlueMotion Comfortline,2012,Dizel,Yarı Otomatik,1598 cc,105 hp,70.000 km,https://www.arabam.com//ilan/sahibinden-satili...,200.000 TL
1,1,15943402,22 Aralık 2020,Volkswagen,Polo,1.4 Comfortline,2013,LPG & Benzin,Düz,1390 cc,85 hp,127.000 km,https://www.arabam.com//ilan/galeriden-satilik...,122.500 TL
2,2,15788492,22 Aralık 2020,Chevrolet,Cruze,1.6 LS,2009,LPG & Benzin,Düz,1598 cc,113 hp,226.000 km,https://www.arabam.com//ilan/galeriden-satilik...,85.000 TL
3,3,16250683,22 Aralık 2020,Renault,Megane,1.5 dCi Touch,2020,Dizel,Yarı Otomatik,1461 cc,115 hp,0 km,https://www.arabam.com//ilan/galeriden-satilik...,277.000 TL
4,4,16250505,22 Aralık 2020,Volkswagen,VW CC,1.4 TSi,2013,Benzin,Yarı Otomatik,1390 cc,160 hp,49.000 km,https://www.arabam.com//ilan/galeriden-satilik...,192.500 TL
5,5,15793297,22 Aralık 2020,Renault,Clio,1.5 dCi Touch,2017,Dizel,Yarı Otomatik,1461 cc,90 hp,80.000 km,https://www.arabam.com//ilan/galeriden-satilik...,154.900 TL
6,6,16249278,22 Aralık 2020,Fiat,Linea,1.3 Multijet Active Plus,2011,Dizel,Düz,1248 cc,90 hp,109.900 km,https://www.arabam.com//ilan/sahibinden-satili...,85.000 TL
7,7,16248951,22 Aralık 2020,Seat,Ibiza,1.0 EcoTSI Style,2017,Benzin,Düz,999 cc,95 hp,37.500 km,https://www.arabam.com//ilan/sahibinden-satili...,155.000 TL
8,8,15293099,22 Aralık 2020,Volkswagen,Passat,1.6 TDi BlueMotion Highline,2015,Dizel,Yarı Otomatik,1598 cc,120 hp,113.000 km,https://www.arabam.com//ilan/galeriden-satilik...,365.000 TL
9,9,16097555,22 Aralık 2020,Volkswagen,Jetta,1.6 TDi Comfortline,2011,Dizel,Yarı Otomatik,1598 cc,105 hp,188.000 km,https://www.arabam.com//ilan/galeriden-satilik...,151.500 TL


In [83]:
df1.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- İlan No:: string (nullable = true)
 |-- İlan Tarihi:: string (nullable = true)
 |-- Marka:: string (nullable = true)
 |-- Seri:: string (nullable = true)
 |-- Model:: string (nullable = true)
 |-- Yıl:: string (nullable = true)
 |-- Yakıt Tipi:: string (nullable = true)
 |-- Vites Tipi:: string (nullable = true)
 |-- Motor Hacmi:: string (nullable = true)
 |-- Motor Gücü:: string (nullable = true)
 |-- Kilometre:: string (nullable = true)
 |-- car_link:: string (nullable = true)
 |-- price: string (nullable = true)



In [84]:
df1.columns

['_c0',
 'İlan No:',
 'İlan Tarihi:',
 'Marka:',
 'Seri:',
 'Model:',
 'Yıl:',
 'Yakıt Tipi:',
 'Vites Tipi:',
 'Motor Hacmi:',
 'Motor Gücü:',
 'Kilometre:',
 'car_link:',
 'price']

In [85]:
for col in df1.columns:
    df1 = df1.withColumn(col, F.ltrim(F.rtrim(df1[col])))

In [86]:
df2 =df1.withColumn("motor_hacmi",F.split('Motor Hacmi:',"-")[0]) \
.withColumn("price",F.split("price"," ")[0]) \
.withColumn("kilometre",F.split("Kilometre:"," ")[0]) \
.drop(F.col('Motor Hacmi:')) 

 


In [87]:
df2=df2.dropna(how="any")

In [88]:
#.withColumn("price",F.split("price",".")[0:2]) \
#.withColumn("kilometre",F.split("Kilometre:",".")[0:2]) \

In [89]:
df3 =df2.withColumn("motor_hacmi",F.split("motor_hacmi"," ")[0]) \
.withColumn("motor_gucu",F.split("Motor Gücü:"," ")[0]) \
.withColumn("marka",F.concat(F.col("Marka:"),F.lit('_'),F.col("Seri:"))) \
.drop(F.col("Motor Gücü:")) \
.drop(F.col("Marka:")) \
.drop(F.col("Seri:")) \
.drop(F.col("_c0")) \
.drop(F.col("İlan Tarihi:"))

In [90]:
df3.limit(2).toPandas()

,İlan No:,Model:,Yıl:,Yakıt Tipi:,Vites Tipi:,Kilometre:,car_link:,price,motor_hacmi,kilometre,motor_gucu,marka
0,16251517,1.6 TDi BlueMotion Comfortline,2012,Dizel,Yarı Otomatik,70.000 km,https://www.arabam.com//ilan/sahibinden-satili...,200.000,1598,70.000,105,Volkswagen_Passat Variant
1,15943402,1.4 Comfortline,2013,LPG & Benzin,Düz,127.000 km,https://www.arabam.com//ilan/galeriden-satilik...,122.500,1390,127.000,85,Volkswagen_Polo


In [91]:
df4=df3.withColumn("price1",F.split(F.col("price"),"\.").getItem(0)) \
.withColumn("price2",F.split(F.col("price"),"\.").getItem(1)) \
.withColumn("price",F.concat(F.col("price1"),F.col("price2")))

In [92]:
df5=df4.withColumn("km1",F.split(F.col("kilometre"),"\.").getItem(0)) \
.withColumn("km2",F.split(F.col("kilometre"),"\.").getItem(1)) \
.withColumn("km",F.concat(F.col("km1"),F.col("km2")))

In [93]:
df6=df5.drop(F.col("km1")) \
.drop(F.col("km2")) \
.drop(F.col("price1")) \
.drop(F.col("price2"))

In [95]:
df6.limit(2).toPandas()

,İlan No:,Model:,Yıl:,Yakıt Tipi:,Vites Tipi:,Kilometre:,car_link:,price,motor_hacmi,kilometre,motor_gucu,marka,km
0,16251517,1.6 TDi BlueMotion Comfortline,2012,Dizel,Yarı Otomatik,70.000 km,https://www.arabam.com//ilan/sahibinden-satili...,200000,1598,70.000,105,Volkswagen_Passat Variant,70000
1,15943402,1.4 Comfortline,2013,LPG & Benzin,Düz,127.000 km,https://www.arabam.com//ilan/galeriden-satilik...,122500,1390,127.000,85,Volkswagen_Polo,127000


In [96]:
df6.printSchema()

root
 |-- İlan No:: string (nullable = true)
 |-- Model:: string (nullable = true)
 |-- Yıl:: string (nullable = true)
 |-- Yakıt Tipi:: string (nullable = true)
 |-- Vites Tipi:: string (nullable = true)
 |-- Kilometre:: string (nullable = true)
 |-- car_link:: string (nullable = true)
 |-- price: string (nullable = true)
 |-- motor_hacmi: string (nullable = true)
 |-- kilometre: string (nullable = true)
 |-- motor_gucu: string (nullable = true)
 |-- marka: string (nullable = true)
 |-- km: string (nullable = true)



In [97]:
df7=df6.select(
F.col('Yıl:').cast("int"),
F.col('Yakıt Tipi:'),
F.col('Vites Tipi:'),
F.col('motor_hacmi').cast("int"),
F.col('motor_gucu').cast("int"),
F.col('km').cast("int"),
F.col('marka'),
F.col('Model:'),
F.col("price").cast("int"),
F.col("car_link:"))

In [98]:
df7.printSchema()

root
 |-- Yıl:: integer (nullable = true)
 |-- Yakıt Tipi:: string (nullable = true)
 |-- Vites Tipi:: string (nullable = true)
 |-- motor_hacmi: integer (nullable = true)
 |-- motor_gucu: integer (nullable = true)
 |-- km: integer (nullable = true)
 |-- marka: string (nullable = true)
 |-- Model:: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- car_link:: string (nullable = true)



In [99]:
df7.limit(3).toPandas()

,Yıl:,Yakıt Tipi:,Vites Tipi:,motor_hacmi,motor_gucu,km,marka,Model:,price,car_link:
0,2012,Dizel,Yarı Otomatik,1598,105,70000,Volkswagen_Passat Variant,1.6 TDi BlueMotion Comfortline,200000,https://www.arabam.com//ilan/sahibinden-satili...
1,2013,LPG & Benzin,Düz,1390,85,127000,Volkswagen_Polo,1.4 Comfortline,122500,https://www.arabam.com//ilan/galeriden-satilik...
2,2009,LPG & Benzin,Düz,1598,113,226000,Chevrolet_Cruze,1.6 LS,85000,https://www.arabam.com//ilan/galeriden-satilik...


In [102]:
# Null check
for(col_name, col_type) in zip(df7.columns, df7.dtypes):
    null_count = df7.filter( (F.col(col_name).isNull()) | (F.col(col_name) == "")).count()
    if(  null_count > 0 ):
        print("{} {} type has {} null values".format(col_name, col_type[1], null_count))

motor_hacmi int type has 3 null values
motor_gucu int type has 3 null values
km int type has 27 null values


In [103]:
df8=df7.dropna(how="any")

In [104]:

categoric_cols = []
numeric_cols = []
discarted_cols = []
label_col = []

In [105]:
# column seperation
for col_name in df8.dtypes:
    if (col_name[0] not in []):
        if (col_name[1] == 'string'):
            categoric_cols.append(col_name[0])
        else: numeric_cols.append(col_name[0])

In [106]:
for col_name in categoric_cols:
    df8.groupBy(col_name).agg(F.count("*").alias("TotalCount")).orderBy(F.desc("TotalCount")).show()

+------------+----------+
| Yakıt Tipi:|TotalCount|
+------------+----------+
|       Dizel|       485|
|LPG & Benzin|       234|
|      Benzin|       196|
|      Hibrit|         1|
+------------+----------+

+-------------+----------+
|  Vites Tipi:|TotalCount|
+-------------+----------+
|          Düz|       535|
|Yarı Otomatik|       221|
|     Otomatik|       160|
+-------------+----------+

+-----------------+----------+
|            marka|TotalCount|
+-----------------+----------+
|       Opel_Astra|        42|
|       Ford_Focus|        41|
|     Renault_Clio|        40|
|Volkswagen_Passat|        40|
|   Toyota_Corolla|        37|
|   Renault_Megane|        36|
|      Honda_Civic|        32|
| Volkswagen_Jetta|        31|
|  Volkswagen_Golf|        30|
|  Renault_Fluence|        29|
|   Renault_Symbol|        25|
|  Volkswagen_Polo|        24|
|        Fiat_Egea|        22|
|       Fiat_Linea|        21|
|     BMW_3 Serisi|        20|
|       Opel_Corsa|        17|
|Mercedes - 

In [107]:
df8.columns

['Yıl:',
 'Yakıt Tipi:',
 'Vites Tipi:',
 'motor_hacmi',
 'motor_gucu',
 'km',
 'marka',
 'Model:',
 'price',
 'car_link:']

In [108]:

# column seperation after analysis
categoric_cols = []
numeric_cols = []
# Why we discard these? Because they have high cardinality. In other words they have too many distinct categories
# Adding them to model will not do good if not too bad.
discarted_cols = ['car_link:']
label_col = ['price']

In [109]:

for col_name in df8.dtypes:
    if (col_name[0] not in discarted_cols+label_col):
        if (col_name[1] == 'string'):
            categoric_cols.append(col_name[0])
        else: numeric_cols.append(col_name[0])

In [110]:
# Here we count the distinct categories in categoric columns. If there is more than two,
# we will add those to to_be_onehotencoded_cols list If there is just 2 we don't need to use onehotencoder
# So if distinct category gt 2 we have to add it to_be_onehotencoded_cols list
to_be_onehotencoded_cols = []

for col_name in categoric_cols:
    count = df8.select(col_name).distinct().count()
    if count > 2:
        to_be_onehotencoded_cols.append(col_name)
    print("{} has {} distinct category.".format(col_name, count))
    

Yakıt Tipi: has 4 distinct category.
Vites Tipi: has 3 distinct category.
marka has 135 distinct category.
Model: has 476 distinct category.


In [111]:
print(to_be_onehotencoded_cols)

['Yakıt Tipi:', 'Vites Tipi:', 'marka', 'Model:']


# StringIndexer

In [112]:
from pyspark.ml.feature import StringIndexer

In [113]:
# Will hold stringIndexer objects and column names
my_dict = {}

# Will collect StringIndexer ojects
string_indexer_objs = []

# Will collect StringIndexer output colnames
string_indexer_output_names = []

# Will collect OneHotEncoder output colnames
ohe_col_input_names = []
ohe_col_output_names = []

for col_name in categoric_cols:
    my_dict[col_name+"_indexobj"] = StringIndexer() \
    .setHandleInvalid('skip') \
    .setInputCol(col_name) \
    .setOutputCol(col_name+"_indexed")
    
    string_indexer_objs.append(my_dict.get(col_name+"_indexobj"))
    string_indexer_output_names.append(col_name+"_indexed")
    if col_name in to_be_onehotencoded_cols:
        ohe_col_input_names.append(col_name+"_indexed")
        ohe_col_output_names.append(col_name+"_ohe")

In [114]:
print(string_indexer_objs)
print(len(string_indexer_objs))

[StringIndexer_36c5d0038c2d, StringIndexer_cba989c8397f, StringIndexer_c6efff858a69, StringIndexer_8cfc7df688d3]
4


In [115]:
print(ohe_col_input_names)
print(len(ohe_col_input_names))

['Yakıt Tipi:_indexed', 'Vites Tipi:_indexed', 'marka_indexed', 'Model:_indexed']
4


In [116]:
print(string_indexer_output_names)
print(len(string_indexer_output_names))

['Yakıt Tipi:_indexed', 'Vites Tipi:_indexed', 'marka_indexed', 'Model:_indexed']
4


# OnehotEncoder

In [117]:
from pyspark.ml.feature import OneHotEncoder

In [118]:
encoder = OneHotEncoder() \
.setInputCols(ohe_col_input_names) \
.setOutputCols(ohe_col_output_names)

In [119]:
from pyspark.ml.feature import VectorAssembler

## VectorAssembler

In [120]:
string_indexer_col_names_ohe_exluded = list(set(string_indexer_output_names).difference(set(ohe_col_input_names)))
string_indexer_col_names_ohe_exluded

[]

In [121]:
if len(categoric_cols) == (len(string_indexer_col_names_ohe_exluded)+len(ohe_col_output_names)):
    print("Columns verified")
else: print("Columns are not verified")

Columns verified


In [122]:
assembler = VectorAssembler().setHandleInvalid("skip") \
.setInputCols(numeric_cols+string_indexer_col_names_ohe_exluded+ohe_col_output_names) \
.setOutputCol('unscaled_features')

In [123]:
label_col

['price']

In [124]:
from pyspark.ml.feature import StandardScaler

In [125]:
scaler = StandardScaler().setInputCol("unscaled_features").setOutputCol("scaled_features")

## PCA

In [126]:
from pyspark.ml.feature import PCA

In [ ]:
#pca = PCA().setInputCol("scaled_features").setK(10).setOutputCol("features")

# Estimator: GBTClassifier

In [128]:
from pyspark.ml.regression import LinearRegression


In [136]:
estimator = LinearRegression() \
.setFeaturesCol("scaled_features") \
.setLabelCol("price")

## Pipeline

In [137]:
from pyspark.ml import Pipeline

In [138]:
pipeline_obj = Pipeline().setStages(string_indexer_objs+[encoder, assembler, scaler, estimator])

# Split data

In [144]:
train_df, test_df = df8.randomSplit([0.05,0.95], seed=142)

# Train MoDEL

In [145]:
pipeline_model = pipeline_obj.fit(train_df)

# Save the model

In [146]:
pipeline_model.stages[-1].write().overwrite().save("saved_models/arabam")

# Prediction

In [147]:
transformed_df = pipeline_model.transform(test_df)

In [148]:
transformed_df.select('price','prediction').show(truncate=False)

+------+-------------------+
|price |prediction         |
+------+-------------------+
|32500 |46340.53537135571  |
|38500 |33294.470848597586 |
|36000 |-108296.10953399539|
|27000 |46938.49000658095  |
|38000 |33688.58072565496  |
|43500 |12423.84226103872  |
|48500 |10385.394679360092 |
|32000 |90534.31282070279  |
|54750 |47883.78772294521  |
|72000 |62290.26080030948  |
|70000 |107786.9873457551  |
|245500|190714.46528116614 |
|156000|92754.1573227942   |
|149900|87220.19096574187  |
|81500 |-14869.190465845168|
|97000 |26146.496539369226 |
|80000 |19950.246545143425 |
|133000|72362.03997519612  |
|99000 |46721.237475045025 |
|87500 |61537.363992482424 |
+------+-------------------+
only showing top 20 rows



# Evaluate the model

In [151]:
from pyspark.ml.evaluation import RegressionEvaluator

In [154]:
evaluator = RegressionEvaluator(labelCol='price')

In [155]:
evaluator.evaluate(transformed_df)

44149.88412794071

In [156]:
evaluator.getMetricName()

'rmse'

# Hyperparameter Tuning

In [158]:
from pyspark.ml.tuning import CrossValidatorModel, ParamGridBuilder, CrossValidator

In [160]:
paramGrid = ParamGridBuilder() \
.addGrid(estimator.regParam, [0.01,0.1,1.0,10.0]) \
.addGrid(estimator.elasticNetParam,[0.0,0.5,1.0]) \
.build()

In [163]:
crossval = CrossValidator(estimator=pipeline_obj,
                          estimatorParamMaps=paramGrid,
                          evaluator=RegressionEvaluator(labelCol='price'),
                          numFolds=5) 

In [164]:
cvModel = crossval.fit(df8)

In [165]:
type(cvModel)

pyspark.ml.tuning.CrossValidatorModel

In [166]:
cvModel.params

[Param(parent='CrossValidatorModel_e98778b93b84', name='estimator', doc='estimator to be cross-validated'),
 Param(parent='CrossValidatorModel_e98778b93b84', name='estimatorParamMaps', doc='estimator param maps'),
 Param(parent='CrossValidatorModel_e98778b93b84', name='evaluator', doc='evaluator used to select hyper-parameters that maximize the validator metric'),
 Param(parent='CrossValidatorModel_e98778b93b84', name='numFolds', doc='number of folds for cross validation'),
 Param(parent='CrossValidatorModel_e98778b93b84', name='seed', doc='random seed.')]

In [167]:
bestModel = cvModel.bestModel

In [168]:
evaluator.evaluate(bestModel.transform(df8))

12617.974378790235

In [169]:
evaluator.evaluate(cvModel.transform(df8))

12617.974378790235

In [170]:
bestModel.write().overwrite().save("saved_models/arabam_best_model")

In [ ]:
spark.stop()